Step 2: Load the Clean, Merged Data
In the first cell, we'll load the output from our previous step. For simplicity, I'll include the merging code again here. You can also save the cleaned DataFrame from notebook 01 to a .csv or .parquet file and load it here.

In [1]:
# Import libraries
import pandas as pd
import numpy as np

# --- Load the datasets ---
# This code is repeated from the last step for completeness.
customers = pd.read_csv('../data/olist_customers_dataset.csv')
orders = pd.read_csv('../data/olist_orders_dataset.csv')
order_items = pd.read_csv('../data/olist_order_items_dataset.csv')
payments = pd.read_csv('../data/olist_order_payments_dataset.csv')
products = pd.read_csv('../data/olist_products_dataset.csv')
translations = pd.read_csv('../data/product_category_name_translation.csv')

# --- Merge the datasets ---
df = pd.merge(orders, customers, on='customer_id')
df = pd.merge(df, order_items, on='order_id')
df = pd.merge(df, products, on='product_id')

# Aggregate payments per order
order_payments_agg = payments.groupby('order_id')['payment_value'].sum().reset_index()
df = pd.merge(df, order_payments_agg, on='order_id')

# Merge with translations
df = pd.merge(df, translations, on='product_category_name', how='left')

# --- Clean and convert data types ---
df['product_category_name_english'].fillna('unknown', inplace=True)
date_columns = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce') # Use coerce to handle any errors

# --- Filter for relevant data ---
# We'll analyze only completed orders for reliable behavioral data.
df_clean = df[df['order_status'] == 'delivered'].copy()

C:\Users\abhis\AppData\Local\Temp\ipykernel_17512\136977078.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['product_category_name_english'].fillna('unknown', inplace=True)


Step 3: Define the Aggregation Level

Our current DataFrame (df_clean) is at the order-item level. We need to aggregate it up to the customer level. The Olist dataset has customer_id (one per order) and customer_unique_id (one per person). We will use customer_unique_id.
Our goal is to create a new DataFrame called customer_df.

Step 4: Engineer RFM Features (Recency, Frequency, Monetary)
This is the classic, powerful framework for customer segmentation.

In [2]:
# --- Engineer RFM Features ---

# Determine the "snapshot date" for our analysis.
# This is the day after the last purchase in the dataset.
snapshot_date = df_clean['order_purchase_timestamp'].max() + pd.Timedelta(days=1)
print(f"Snapshot date: {snapshot_date}")

# Group by customer and aggregate
customer_df = df_clean.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': lambda x: (snapshot_date - x.max()).days, # Recency
    'order_id': 'nunique',                                                # Frequency
    'payment_value': 'sum'                                                # Monetary
})

# Rename the columns for clarity
customer_df.rename(columns={
    'order_purchase_timestamp': 'recency',
    'order_id': 'frequency',
    'payment_value': 'monetary'
}, inplace=True)

# Display the first few rows of our new customer-level dataframe
customer_df.head()

Snapshot date: 2018-08-30 15:00:37


,recency,frequency,monetary
customer_unique_id,,,
0000366f3b9a7992bf8c76cfdf3221e2,112,1,141.90
0000b849f77a49e4a4ce2b2a4ca5be3f,115,1,27.19
0000f46a3911fa3c0805444483337064,537,1,86.22
0000f6ccb0745a6a4b88665a16c9f078,321,1,43.62
0004aac84e0df4da2b147fca70cf8255,288,1,196.89


Critical Thinking: Why do we use a snapshot_date? Because Recency needs a reference point. It measures how many days from a specific point in time a customer last made a purchase.

Step 5: Engineer Advanced Behavioral Features
To make our model truly insightful, let's go beyond RFM. We'll add features that describe the nature of their purchasing habits.

In [3]:
# --- Engineer Advanced Features ---

# We can perform another aggregation and merge it
advanced_features = df_clean.groupby('customer_unique_id').agg({
    'payment_value': 'mean',                                    # Average order value
    'product_category_name_english': 'nunique',                 # Product diversity
    'order_purchase_timestamp': ['min', 'max']                  # First and last purchase dates
}).reset_index()

# Flatten the multi-level column names
advanced_features.columns = ['customer_unique_id', 'avg_order_value', 'product_diversity', 'first_purchase_date', 'last_purchase_date']

# Calculate Customer Tenure
advanced_features['tenure'] = (advanced_features['last_purchase_date'] - advanced_features['first_purchase_date']).dt.days

# Merge these new features into our main customer_df
customer_df = pd.merge(customer_df, advanced_features[['customer_unique_id', 'avg_order_value', 'product_diversity', 'tenure']], on='customer_unique_id')

customer_df.head()

,customer_unique_id,recency,frequency,monetary,avg_order_value,product_diversity,tenure
0,0000366f3b9a7992bf8c76cfdf3221e2,112,1,141.90,141.90,1,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,115,1,27.19,27.19,1,0
2,0000f46a3911fa3c0805444483337064,537,1,86.22,86.22,1,0
3,0000f6ccb0745a6a4b88665a16c9f078,321,1,43.62,43.62,1,0
4,0004aac84e0df4da2b147fca70cf8255,288,1,196.89,196.89,1,0


Step 6: Final Cleanup and Sanity Check
Let's inspect our final customer-level DataFrame.

In [4]:
# --- Final Inspection ---

# Check the descriptive statistics of our new features
customer_df.describe()

,recency,frequency,monetary,avg_order_value,product_diversity,tenure
count,93357.000000,93357.000000,93357.000000,93357.000000,93357.000000,93357.000000
mean,237.936673,1.033420,211.833718,160.450078,1.026040,2.634061
std,152.584315,0.209099,642.166523,219.725771,0.171621,24.955954
min,1.000000,1.000000,9.590000,9.590000,1.000000,0.000000
25%,114.000000,1.000000,63.760000,62.390000,1.000000,0.000000
50%,219.000000,1.000000,112.950000,105.700000,1.000000,0.000000
75%,346.000000,1.000000,201.740000,176.760000,1.000000,0.000000
max,695.000000,15.000000,109312.640000,13664.080000,5.000000,633.000000


You might notice something interesting in the .describe() output: frequency might have a max value far greater than the others, and monetary will have a very wide range. This is exactly why we will need to scale the data in the next phase before clustering.

In [5]:
# At the end of your notebook, save the result
customer_df.to_csv('../data/customer_features.csv')